In [1]:
import os
import gzip
import matplotlib.pyplot as plt
import sklearn as sk
import time
import numpy as np
import tensorflow
import cv2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras import regularizers
from tensorflow.keras import Model, Input, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from ipywidgets import interact

In [2]:
proxy_1 = 'http://proxy-dmz.intel.com/911'
proxy_2 = 'http://proxy-chain.intel.com:911'

os.environ['http_proxy'] = proxy_1
os.environ['HTTP_PROXY'] = proxy_1
os.environ['https_proxy'] = proxy_1
os.environ['HTTPS_PROXY'] = proxy_1

In [3]:
device_name = tensorflow.test.gpu_device_name()
if not device_name:
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
def load_mnist(path, kind, **kwargs):
    """
    Load the MNIST dataset from a defined directory.
    Args:
        path: the directory to the MNIST dataset, for example 'data/mnist'
        kind: type of data that is either train or test.
    Returns:
        images, labels: the arrays of MNIST dataset.
    
    """
    labels_path = os.path.join(path,'%s-labels-idx1-ubyte.gz'% kind)
    images_path = os.path.join(path,'%s-images-idx3-ubyte.gz'% kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype = np.uint8, offset = 8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype = np.uint8, offset = 16).reshape(len(labels), 784)

    return images, labels


def resize_mnist(imgs, original_size, new_size, *args):
    """
    Return the resized array of MNIST images with a new resolution that is compatible with several CNNs such as VGG19, etc.
    
    Args:
        imgs: np.ndarray
            MNIST DATASET.
        original_size : tuple of shape (height, width)
            DIMENSION OF EACH MNIST IMAGE.
        new_size : tuple of shape (height, width)
            USER-DEFINED DIMENSION OF EACH MNIST IMAGE
    Returns:
        tmp: np.ndarray
            RESIZED MNIST DATASET
    """
    tmp = np.empty((imgs.shape[0], new_size[0], new_size[1]))
    
    for i in range(len(imgs)):
        img = imgs[i].reshape(original_size[0], original_size[1]).astype('uint8')
        img = cv2.resize(img, new_size)
        img = img.astype('float32') / 255
        tmp[i] = img
        
    return tmp

def custom_VGG19(input_shape):
    convolutional_base = VGG19(weights = None, include_top = False, input_shape = input_shape)
    convolutional_base.trainable = True
    
    model = Sequential()
    model.add(convolutional_base)
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(4096, activation = 'relu', kernel_regularizer = regularizers.l1_l2(l1 = 1e-5, l2 = 1e-4),
          bias_regularizer = regularizers.l2(1e-4),
          activity_regularizer = regularizers.l2(1e-5))(x)
    x = Dense(2048, activation = 'relu', kernel_regularizer = regularizers.l1_l2(l1 = 1e-5, l2 = 1e-4),
          bias_regularizer = regularizers.l2(1e-4),
          activity_regularizer = regularizers.l2(1e-5))(x)
    x = Dense(128, activation = 'relu', kernel_regularizer = regularizers.l1_l2(l1 = 1e-5, l2 = 1e-4),
          bias_regularizer = regularizers.l2(1e-4),
          activity_regularizer = regularizers.l2(1e-5))(x)
    out = Dense(10, activation = 'softmax')(x)
    final_model = Model(inputs = model.input, outputs = out)
    
    opt = Adam(learning_rate = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-07)
    final_model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy', 'Precision', 'Recall'])
    final_model.summary()
    return final_model
    

In [5]:
X_train, y_train = load_mnist('fashion', kind = 'train')
X_test, y_test = load_mnist('fashion', kind = 't10k')


X_train = resize_mnist(X_train, (28,28), (32,32))
X_train = np.stack((X_train,) * 3, axis = -1)
X_test = resize_mnist(X_test, (28,28), (32,32))
X_test = np.stack((X_test,) * 3, axis = -1)

y_train = to_categorical(y_train, num_classes = 10)
y_test = to_categorical(y_test, num_classes = 10)

@interact (n = (0, X_train.shape[0]))
def display_image(n = 0):
    plt.imshow(X_train[n,:], cmap = 'gray')
    plt.show()
    return None
           

interactive(children=(IntSlider(value=0, description='n', max=60000), Output()), _dom_classes=('widget-interac…

In [6]:
model = custom_VGG19((32,32,3))
history = model.fit(X_train, y_train, epochs = 100, verbose = 1, batch_size = 128,
                         shuffle = True, validation_data = (X_test, y_test))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19_input (InputLayer)     [(None, 32, 32, 3)]       0         
_________________________________________________________________
vgg19 (Model)                (None, 1, 1, 512)         20024384  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 4096)              2101248   
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              8390656   
_________________________________________________________________
dense_2 (Dense)              (None, 128)               262272    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290  

KeyboardInterrupt: 